In [ ]:
import sys
import json
import tifffile
import collections
from matplotlib import pyplot as plt

In [ ]:
sys.path.insert(0, '../../')
import opencell

In [ ]:
from opencell.imaging.images import RawPipelineTIFF

In [ ]:
filepath = '/Users/keith.cheveralls/image-data/MMStack_1691-H8-35.ome.tif'
filepath = '/Users/keith.cheveralls/image-data/H6_10_PCNA.ome.tif'

In [ ]:
filepath = '/Users/keith.cheveralls/image-data/MMStack_673-D7-25.ome.tif'

In [ ]:
tiff = RawPipelineTIFF(filepath)
tiff.parse_micromanager_metadata()
tiff.validate_micromanager_metadata()
tiff.split_channels()

In [ ]:
tiff = tifffile.TiffFile(filepath)
len(tiff.pages)

In [ ]:
# MicroManager tag attributes
mm_tag = {
    'code': 'MicroManagerMetadata',
    'dtype': 's',
    'count': 0,
    'value': None,
    'writeonce': False
}

# the order matters (because the tag is passed to TiffWriter.save as a tuple)
mm_tag = collections.OrderedDict(**mm_tag)

downsample_by = 10
pages_out = []
tags_out = []
for ind, page in enumerate(tiff.pages):
    # use the mm tag from the first page for every page
    mm_tag['value'] = json.dumps(tiff.pages[0].tags['MicroManagerMetadata'].value)
    tags_out.append(tuple(mm_tag.values()))
    pages_out.append(page.asarray()[::downsample_by, ::downsample_by])

In [ ]:
tiff_out = tifffile.TiffWriter('/Users/keith.cheveralls/image-data/test_raw_stack.tif')
for page, tag in zip(pages_out, tags_out):
    tiff_out.save(page, extratags=[tag], contiguous=False)
tiff_out.close()

In [ ]:
tiff = tifffile.TiffFile('/Users/keith.cheveralls/image-data/test_raw_stack.tif')
len(tiff.pages)

In [ ]:
tiff.pages[1].tags['MicroManagerMetadata'].value['SliceIndex']

In [ ]:
tiff = RawPipelineTIFF('/Users/keith.cheveralls/image-data/test_raw_stack.tif')
tiff.parse_micromanager_metadata()
tiff.validate_micromanager_metadata()
tiff.split_channels()
tiff.did_split_channels

In [ ]:
tiff.project_stack('405', axis='z')

In [ ]:
tiff.calculate_z_profiles('405')

In [ ]:
stacks, result = tiff.align_cell_layer(-5, 5, .5)

In [ ]:
result

In [ ]:
tiff.events